# Demostración: CSV vs. Parquet

En esta demostración, vamos a explorar las diferencias estructurales y de rendimiento entre el formato CSV tradicional y el formato columnar optimizado Apache Parquet.

**Objetivos:**
1. Visualizar la estructura de un archivo CSV.
2. Convertir los datos a formato Parquet.
3. Demostrar el concepto clave de "lectura selectiva de columnas" (Column Pruning) que hace a Parquet tan eficiente para Big Data.

In [19]:
import pandas as pd
import os

# Nombre del archivo CSV
nombre_csv = 'ventas_grandes.csv'

# Leemos el archivo CSV y lo cargamos en un DataFrame de pandas
df = pd.read_csv(nombre_csv)

print(f"Se cargaron {len(df)} filas desde el archivo '{nombre_csv}'.")
print("Mostrando las primeras 5 filas:")
df.head()

Se cargaron 1000000 filas desde el archivo 'ventas_grandes.csv'.
Mostrando las primeras 5 filas:


,ID_Venta,ID_Producto,Producto,Categoria,Precio_Unitario,Cantidad,Total_Venta,Fecha_Venta,Ciudad,Metodo_Pago
0,10001,PROD_010,Mochila para Laptop,Accesorios,44.82,5,224.10,2024-07-10 21:58:45,Madrid,Tarjeta de Credito
1,10002,PROD_010,Mochila para Laptop,Accesorios,47.67,2,95.34,2025-06-23 09:58:45,Valencia,Bizum
2,10003,PROD_001,Laptop Pro X1,Electronica,1253.10,3,3759.30,2024-09-10 13:58:45,Sevilla,PayPal
3,10004,PROD_010,Mochila para Laptop,Accesorios,46.27,3,138.81,2024-08-17 04:58:45,Valencia,Tarjeta de Credito
4,10005,PROD_010,Mochila para Laptop,Accesorios,46.69,5,233.45,2024-12-23 12:58:45,Zaragoza,Tarjeta de Credito


## Paso 2: Convertir a Parquet

Ahora, vamos a guardar este DataFrame en formato Parquet. Este formato es binario y columnar, optimizado para el análisis de grandes volúmenes de datos.

In [20]:
# Nombre del archivo Parquet
nombre_parquet = 'ventas_grandes.parquet'

# Guardamos el DataFrame en formato Parquet
df.to_parquet(nombre_parquet)

print(f"Archivo '{nombre_parquet}' creado con éxito.")

Archivo 'ventas_grandes.parquet' creado con éxito.


## Paso 3: Comparación de Tamaño de Archivo

Una de las primeras ventajas de Parquet es su eficiencia en el almacenamiento gracias a la compresión avanzada. Comparemos los tamaños de ambos archivos.

In [21]:
# Obtenemos los tamaños de los archivos en bytes y los convertimos a KB
tamaño_csv_kb = os.path.getsize(nombre_csv) / 1024
tamaño_parquet_kb = os.path.getsize(nombre_parquet) / 1024

print(f"Tamaño del archivo CSV: {tamaño_csv_kb:.2f} KB")
print(f"Tamaño del archivo Parquet: {tamaño_parquet_kb:.2f} KB")
print(f"El archivo Parquet es un {((tamaño_csv_kb - tamaño_parquet_kb) / tamaño_csv_kb) * 100:.2f}% más pequeño.")

Tamaño del archivo CSV: 98279.27 KB
Tamaño del archivo Parquet: 16716.75 KB
El archivo Parquet es un 82.99% más pequeño.


## Paso 4: La Magia de la Lectura Selectiva (Column Pruning)

Aquí está la ventaja más importante de la estructura columnar de Parquet. Si solo necesitamos analizar algunas columnas, podemos pedírselo explícitamente y el motor solo leerá esos datos del disco, ignorando el resto.

In [25]:
# Definimos las columnas que nos interesan
columnas_deseadas = ['Producto', 'Precio_Unitario', 'Ciudad']

# Leemos SOLO esas columnas desde el archivo Parquet
df_selectivo = pd.read_parquet(nombre_parquet, columns=columnas_deseadas)

print(f"Se leyeron solo {len(columnas_deseadas)} columnas del archivo Parquet.")
print("Mostrando las primeras 5 filas del DataFrame resultante:")
df_selectivo.head()

Se leyeron solo 3 columnas del archivo Parquet.
Mostrando las primeras 5 filas del DataFrame resultante:


,Producto,Precio_Unitario,Ciudad
0,Mochila para Laptop,44.82,Madrid
1,Mochila para Laptop,47.67,Valencia
2,Laptop Pro X1,1253.10,Sevilla
3,Mochila para Laptop,46.27,Valencia
4,Mochila para Laptop,46.69,Zaragoza


## Paso 5: ¿Y podemos hacer lectura selectiva en un CSV?

Ahora que sabemos que Parquet es eficiente, surge una pregunta: ¿podemos lograr el mismo efecto de leer solo unas pocas columnas directamente desde nuestro archivo CSV original? Vamos a intentarlo.

In [24]:
# Definimos las mismas columnas que queremos
columnas_deseadas = ['Producto', 'Precio_Unitario', 'Ciudad']

# Intentamos leer solo esas columnas desde el archivo CSV
print(f"Pidiendo a Pandas que lea solo las columnas: {columnas_deseadas} desde el CSV...")
df_csv_selectivo = pd.read_csv(nombre_csv, usecols=columnas_deseadas)

print("\n¡Lectura completada! Mostrando las primeras 5 filas del resultado:")
df_csv_selectivo.head()

Pidiendo a Pandas que lea solo las columnas: ['Producto', 'Precio_Unitario', 'Ciudad'] desde el CSV...

¡Lectura completada! Mostrando las primeras 5 filas del resultado:


,Producto,Precio_Unitario,Ciudad
0,Mochila para Laptop,44.82,Madrid
1,Mochila para Laptop,47.67,Valencia
2,Laptop Pro X1,1253.10,Sevilla
3,Mochila para Laptop,46.27,Valencia
4,Mochila para Laptop,46.69,Zaragoza


### La Diferencia Fundamental (Lo que no se ve)

Como vemos, ¡el resultado es el mismo! Pandas nos devolvió un DataFrame solo con las columnas que pedimos. Entonces, ¿dónde está la ventaja de Parquet?

La diferencia clave está en **CÓMO** se obtuvieron esos datos:

*   **Para el CSV:** Debido a su estructura basada en filas, `pandas` (y cualquier otra herramienta) **debe leer y procesar el archivo completo línea por línea** para encontrar las columnas que necesita. Carga todo en memoria y *luego* descarta las columnas que no le pedimos. Para archivos pequeños, esto es instantáneo. Para un archivo de 100 GB, tendría que leer los 100 GB del disco, lo cual sería extremadamente lento.

*   **Para el Parquet:** Como vimos antes, su estructura columnar permite a la herramienta **ir directamente a los bloques de datos de las columnas específicas en el disco**, ignorando por completo el resto. Para un archivo de 100 GB, si las columnas que queremos solo ocupan 5 GB, solo se leen esos 5 GB del disco.

**Esta es la diferencia entre "filtrar después de cargar todo" (CSV) y "cargar solo lo que se necesita" (Parquet).**